In [122]:
#importing libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [123]:
#url to scrap page
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 

soup = bs(r.content, 'html5lib') 

In [124]:
Geospatial_Data=pd.read_csv('../data/Geospatial_Coordinates.csv')

In [125]:
DataFrameHeader=[]
DataFrameObject={}
#loop over table row followed by table data to select data
for row in soup("tr"):
    for i,cell in enumerate(row("th",{'class':None})):
        DataFrameHeader.append(cell.text.replace('\n',''))
        DataFrameObject[cell.text.replace('\n','')]=[]
            
    for i,cell in enumerate(row("td",{'style':None})):
        if cell.text:
            DataFrameObject[DataFrameHeader[i]].append(cell.text.replace('\n',''))

df= pd.DataFrame(DataFrameObject) 
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [126]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head(5)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [127]:
#cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
#df = df.apply(assignNotassigned)
df['Neighborhood'] = np.where(df.Neighborhood == 'Not assigned'\
                                       ,df.Borough,df.Neighborhood)




In [128]:
#More than one neighborhood can exist in one postal code area
#combined into one row with the neighborhoods separated with a comma

dfNew = df.groupby('Postcode').agg({'Borough' :'first',
                                     'Neighborhood': ', '.join,
                                     }).reset_index()
dfNew.rename(columns={'Postcode':"Postal Code"},inplace=True)

In [129]:
dfNew.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [130]:
dfNew.shape

(103, 3)

#### using geospatial data to map latitude and longitude to respective postal code

In [131]:
Geospatial_Data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging two data frame based on Postal code

In [132]:
dfNew = pd.merge(dfNew, Geospatial_Data, on='Postal Code', how='left')

In [134]:
display(dfNew.head())

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [135]:
dfNew.shape

(103, 5)

In [136]:
# double check on the dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfNew['Borough'].unique()),
        dfNew.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.
